## Imports and Common Functions

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
def read_image():  
  img = cv2.imread('/content/Fig0441(a)(characters_test_pattern).tif')
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img = cv2.resize(img, (200, 200))
  return img

In [ ]:
def to_fourier(img):
  return np.fft.fftshift(np.fft.fft2(img))

In [ ]:
def to_spatial(img):
  return np.fft.ifft2(np.fft.ifftshift(img))

In [ ]:
def apply_filter(img, filter):
  img = to_fourier(img)
  img = img * filter
  return to_spatial(img).real

## Ideal Low Pass Filter

In [ ]:
def ideal_filter_low(img, cutoff):
  m, n = img.shape
  kernel = np.zeros((m, n), dtype=float)
  center = m // 2, n // 2

  for i in range(m):
    for j in range(n):
      distance = math.sqrt((center[0] - i) ** 2 + (center[1] - j) ** 2)
      if distance <= cutoff:
        kernel[i][j] = 1
  
  return kernel

In [ ]:
img = read_image()
cv2_imshow(img)

In [ ]:
filter = ideal_filter_low(img, cutoff=30)
plt.imshow(filter, cmap = 'gray')

In [ ]:
result = apply_filter(img, ideal_filter_low(img, cutoff=10))
cv2_imshow(result)
result = apply_filter(img, ideal_filter_low(img, cutoff=30))
cv2_imshow(result)
result = apply_filter(img, ideal_filter_low(img, cutoff=50))
cv2_imshow(result)

For the Ideal Low Pass Filter, increasing the cutoff frequency makes the image clearer. The ringing effect also decreases. For very low cutoff frequencies, the ringing effect is so high that it is no longer distinguishable as the ringing effect.

## Butterworth Low Pass Filter

In [ ]:
def butterworth_filter_low(img, cutoff, order):
  m, n = img.shape
  kernel = np.zeros((m, n), dtype=float)
  center = m // 2, n // 2

  for i in range(m):
    for j in range(n):
      distance = math.sqrt((center[0] - i) ** 2 + (center[1] - j) ** 2 )
      kernel[i][j] = 1 / (1 + (distance / cutoff) ** (2 * order))
  
  return kernel

In [ ]:
img = read_image()
filter = butterworth_filter_low(img, cutoff=30, order=2)
plt.imshow(filter, cmap = 'gray')

In [ ]:
result = apply_filter(img, butterworth_filter_low(img, cutoff=30, order=2))
cv2_imshow(result)
result = apply_filter(img, butterworth_filter_low(img, cutoff=30, order=75))
cv2_imshow(result)
result = apply_filter(img, butterworth_filter_low(img, cutoff=100, order=2))
cv2_imshow(result)

For the Butterworth low pass filter, a low value for the order ensures that there is no ringing effect. We can see that the image simply gets blurrier with lower cutoff frequencies. If we increase the order to a high value, we will see the ringing effect.

## Gaussian Low Pass Filter

In [ ]:
def gaussian_filter_low(img, cutoff):
  m, n = img.shape
  kernel = np.zeros((m, n), dtype=float)
  center = m // 2, n // 2

  for i in range(m):
    for j in range(n):
      distance = (center[0] - i) ** 2 + (center[1] - j) ** 2
      kernel[i][j] = math.exp(-distance / (2 * (cutoff ** 2)))
  
  return kernel

In [ ]:
img = read_image()
filter = gaussian_filter_low(img, cutoff=30)
plt.imshow(filter, cmap = 'gray')

In [ ]:
result = apply_filter(img, gaussian_filter_low(img, cutoff=10))
cv2_imshow(result)
result = apply_filter(img, gaussian_filter_low(img, cutoff=30))
cv2_imshow(result)
result = apply_filter(img, gaussian_filter_low(img, cutoff=50))
cv2_imshow(result)

The Gaussian Low Pass filter removes the order parameter that is present in the Butterworth Low Pass filter. As such, there is no possibility of a ringing effect. We just see the image become increasingly blurry as the cutoff frequency decreases.

## Ideal High Pass Filter

In [ ]:
def ideal_filter_high(img, cutoff):
  m, n = img.shape
  kernel = np.ones((m, n), dtype=float)
  center = m // 2, n // 2

  for i in range(m):
    for j in range(n):
      distance = math.sqrt((center[0] - i) ** 2 + (center[1] - j) ** 2)
      if distance <= cutoff:
        kernel[i][j] = 0
  
  return kernel

In [ ]:
img = read_image()
filter = ideal_filter_high(img, cutoff=30)
plt.imshow(filter, cmap = 'gray')

In [ ]:
result = apply_filter(img, ideal_filter_high(img, cutoff=10))
cv2_imshow(result)
result = apply_filter(img, ideal_filter_high(img, cutoff=30))
cv2_imshow(result)
result = apply_filter(img, ideal_filter_high(img, cutoff=50))
cv2_imshow(result)

Similar to the low pass variant, the Ideal High Pass Filter also shows an increased ringing effect as the cutoff frequency decreases. Since this is a high pass filter, a higher cutoff frequency gives us clearer edges as most of the information from the smooth regions is removed.

## Butterworth High Pass Filter

In [ ]:
def butterworth_filter_high(img, cutoff, order):
  m, n = img.shape
  kernel = np.zeros((m, n), dtype=float)
  center = m // 2, n // 2

  for i in range(m):
    for j in range(n):
      distance = ((center[0] - i) ** 2 + (center[1] - j) ** 2 ) ** 0.5
      kernel[i][j] = 1 / (1 + (cutoff / (distance + 1)) ** (2 * order))
  
  return kernel

In [ ]:
img = read_image()
filter = butterworth_filter_high(img, cutoff=30, order=2)
plt.imshow(filter, cmap = 'gray')

In [ ]:
result = apply_filter(img, butterworth_filter_high(img, cutoff=30, order=2))
cv2_imshow(result)
result = apply_filter(img, butterworth_filter_high(img, cutoff=30, order=75))
cv2_imshow(result)
result = apply_filter(img, butterworth_filter_high(img, cutoff=80, order=2))
cv2_imshow(result)

For the Butterworth High Pass Filter, we see more distinctive edges as the cutoff frequency is increased. Having a low value for the order parameter ensures that there is no ringing effect. However, increasing the order introduces the effect.

## Gaussian High Pass Filter

In [ ]:
def gaussian_filter_high(img, cutoff):
  m, n = img.shape
  kernel = np.zeros((m, n), dtype=float)
  center = m // 2, n // 2

  for i in range(m):
    for j in range(n):
      distance = (center[0] - i) ** 2 + (center[1] - j) ** 2
      kernel[i][j] = 1 - math.exp(-distance / (2 * (cutoff ** 2)))
  
  return kernel

In [ ]:
img = read_image()
filter = gaussian_filter_high(img, cutoff=30)
plt.imshow(filter, cmap = 'gray')

In [ ]:
result = apply_filter(img, gaussian_filter_high(img, cutoff=10))
cv2_imshow(result)
result = apply_filter(img, gaussian_filter_high(img, cutoff=30))
cv2_imshow(result)
result = apply_filter(img, gaussian_filter_high(img, cutoff=50))
cv2_imshow(result)

Since there is no order parameter, the Guassian High Pass Filter cannot cause any ringing effect. The edges simply become more distinctive as the cutoff frequency is increased.